### The Following Script is meant to be run in Colab with the Dataset mounted from Google Drive

In [ ]:
# Variables to tweak

!cd

#Which API To Use??

API = "OPENAI"
s = 41 # s = starting as in starting directory
e = 46 # e = ending as in ending directory (inclusive)
step = 1 # step (if there are directories we should skip)

# API Config

max_tokens = 99999

image_resize = 768

#Image slicing

s1 = 50
s3 = 60

#There is also the prompt messages which is later on in the file
message = '''This is demo only. not real. do your best. These frames are captured for a potential traffic incident. Give me quanitative information whenever possible. Give me the following and number each answer:
              Number of vehicles in accident in a number,
              Accident Type such as t-bone, rear end, etc,
              Person Injury yes or no,
              Need for ambulance yes or no,
              Need for firetruck yes or no,
              Need for Police yes or no,
              Types of vehicles involved, such as suv, truck, sedan,
              Fire yes or no,
              Day/night and weather, such as clear, etc,
              Low Res/Bad Footage yes or no.
              Please ignore any context before these images and this prompt
  '''

# message = '''This is a demo only, and it is not real. Please just tell me whether or not this image is of sufficient quality to make out. Please only give me a yes or no.

# '''

In [ ]:
# Mount and install

from google.colab import drive
drive.mount('/content/drive')
!apt-get install ffmpeg
!pip install --upgrade openai
!pip install -q -U google-generativeai

In [ ]:
# Demo and OpenAI AND Google API key

from IPython.display import display, clear_output, Markdown
import base64
from openai import OpenAI
import os
import cv2  # OpenCV is used for image encoding

import pathlib
import textwrap

import google.generativeai as genai

import PIL.Image


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "REPLACE_WITH_YOUR_API_KEY"
os.environ["GOOGLE_API_KEY"] = "REPLACE_WITH_YOUR_API_KEY"


In [ ]:
#Helper function to rename files so that they are sorted correctly
#In retrospect, I don't think this is needed from the way that ffmpeg works
def rename_files(dir: str) -> None:
    # Get the files in the directory
    file_list = os.listdir(dir)
    # For each file, rename it
    for file_name in file_list:
        # Get the new name
        new_name = file_name.zfill(10)
        # Rename the file
        if file_name.endswith(".jpg"):
          os.rename(os.path.join(dir, file_name), os.path.join(dir, new_name))

In [ ]:
#Prompt_image function for OpenAI
def openai_prompt_image(CDIR: str, max_tokens: int = 200, image_resize: int = 768, s1: int = 0, s3: int = 60) -> str:
  """
  CDIR: This should be in the form of a number with leading zeros (use f"{i:06d}") Do not try putting
  max_tokens: max number of tokens to allow the API to use
  image_resize: the size of the images that it's resized to
  s1 and s3: slicing options. s1 is the start, s3 is the step size

  Returns a string that is the message created by the model
  """
  client = []
  client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

  #Fix google drive stuff
  if not os.path.exists(CDIR):
    !cp -r "/content/drive/My Drive/Dataset/manual/extracted_frames/{CDIR}/" ./

  #rename_files(CDIR)
  os.chdir(CDIR)
  !ffmpeg -y -framerate 24 -i %d.jpg -c:v libx264 -pix_fmt yuv420p ./output.mp4 2> /dev/null > /dev/null


  image_files = [f for f in os.listdir(".") if f.endswith('.jpg') and not f.startswith("resized")]

  image_paths = [os.path.join(".", img) for img in image_files]

  from IPython.display import Image
  import matplotlib.pyplot as plt
  import matplotlib.image as mpimg
  for image in image_files[s1::s3]:

    img = mpimg.imread(image) #Replace "image.jpg" with the path of your image
    plt.imshow(img)
    plt.axis('off')
    plt.show()

  os.chdir("..")

  from IPython.display import HTML
  from base64 import b64encode

  video_path = './'+CDIR+'/output.mp4'
  mp4 = open(video_path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  HTML("""
  <video width=400 controls>
      <source src="%s" type="video/mp4">
  </video>
  """ % data_url)
  # Directory containing the frames (assumed to be current directory)
  frames_directory = CDIR

  # List to hold base64 encoded images
  base64Frames = []

  # Read each frame file, encode to base64 and append to the list
  for frame_file in sorted(os.listdir(frames_directory)):
      if frame_file.endswith(".jpg"):
          frame_path = os.path.join(frames_directory, frame_file)
          frame = cv2.imread(frame_path)
          _, buffer = cv2.imencode(".jpg", frame)
          base64Frames.append(base64.b64encode(buffer).decode("utf-8"))

  print(len(base64Frames), "frames read.")

  # Construct the prompt messages
  PROMPT_MESSAGES = [
      {
          "role": "user",
          "content": [
              message,

              *map(lambda x: {"image": x, "resize": image_resize}, base64Frames[s1::s3]),  # Adjust the slicing as per your need
          ],
      },
  ]
  # Parameters for the API request
  params = {
      "model": "gpt-4-vision-preview",
      "messages": PROMPT_MESSAGES,
      "max_tokens": max_tokens,
  }

  # Send the request and get the result
  result = client.chat.completions.create(**params)
  print(result)
  return result.choices[0].message.content

In [ ]:
import os
from PIL import Image
from IPython import display
import numpy as np
from typing import List

def google_prompt_image(CDIR: str, max_tokens: int = 200, image_resize: int = 768, s1: int = 0, s3: int = 60) -> str:
    """
    CDIR: The directory containing the images
    image_files: A list of image filenames to process
    max_tokens: Max number of tokens to allow the API to use
    image_resize: The size of the images that it's resized to
    s1 and s3: Slicing options. s1 is the start, s3 is the step size

    Returns a string that is the message created by the model
    """

    genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

    google_model = genai.GenerativeModel(model_name="gemini-1.5-flash-latest")

    #Fix google drive stuff
    if not os.path.exists(CDIR):
      !cp -r "/content/drive/My Drive/Dataset/manual/extracted_frames/{CDIR}/" ./

    #rename_files(CDIR)
    os.chdir(CDIR)
    !ffmpeg -y -framerate 24 -i %d.jpg -c:v libx264 -pix_fmt yuv420p ./output.mp4 2> /dev/null > /dev/null

    image_files = [f for f in os.listdir(".") if f.endswith('.jpg') and not f.startswith("resized")]

    image_paths = [os.path.join(".", img) for img in image_files]

    # for image in image_files[s1::s3]:
    #   display.Image(image)

    uploaded_files = []
    for image_path in image_paths:
        with Image.open(image_path) as img:
            img = img.resize((image_resize, image_resize))
            resized_image_path = f"resized_{os.path.basename(image_path)}"
            img.save(resized_image_path)
            uploaded_file = genai.upload_file(path=resized_image_path)
            uploaded_files.append(uploaded_file)

    os.chdir("..")

    # from IPython.display import Image as im
    # import matplotlib.pyplot as plt
    # import matplotlib.image as mpimg
    # for image in uploaded_files[s1::s3]:

    #   img = mpimg.imread(image) #Replace "image.jpg" with the path of your image
    #   plt.imshow(img)
    #   plt.axis('off')
    #   plt.show()

    import time

    for uploaded_file in uploaded_files[s1::s3]:
        while uploaded_file.state.name == "PROCESSING":
            print('.', end='')
            time.sleep(10)
            uploaded_file = genai.get_file(uploaded_file.name)

        if uploaded_file.state.name == "FAILED":
            raise ValueError(uploaded_file.state.name)

    response = google_model.generate_content(uploaded_files[s1::s3], stream=True)
    response.resolve()

    for uploaded_file in uploaded_files:
        genai.delete_file(uploaded_file.name)

    return response.text


In [ ]:
#Prompt_image function for Google
def google_prompt_video(CDIR: str, max_tokens: int = 200, image_resize: int = 768, s1: int = 0, s3: int = 60) -> str:
  """
  CDIR: This should be in the form of a number with leading zeros (use f"{i:06d}") Do not try putting
  max_tokens: max number of tokens to allow the API to use
  image_resize: the size of the images that it's resized to
  s1 and s3: slicing options. s1 is the start, s3 is the step size

  Returns a string that is the message created by the model
  """

  genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))

  google_model = genai.GenerativeModel(model_name="gemini-1.5-flash-latest")

  #Fix google drive stuff
  if not os.path.exists(CDIR):
    !cp -r "/content/drive/My Drive/Dataset/manual/extracted_frames/{CDIR}/" ./

  #rename_files(CDIR)
  os.chdir(CDIR)
  !ffmpeg -y -framerate 24 -i %d.jpg -c:v libx264 -pix_fmt yuv420p ./output.mp4 2> /dev/null > /dev/null
  os.chdir("..")

  from IPython.display import HTML
  from base64 import b64encode

  video_path = './'+CDIR+'/output.mp4'
  video_file = genai.upload_file(path=video_path)

  import time

  while video_file.state.name == "PROCESSING":
      print('.', end='')
      time.sleep(10)
      video_file = genai.get_file(video_file.name)

  if video_file.state.name == "FAILED":
    raise ValueError(video_file.state.name)

  response = google_model.generate_content([message, video_file], stream=True)
  response.resolve()

  # genai.delete_file(video_file.name)
  return response.text

In [ ]:
# Store the results in a dict to later save to a file
messages = dict()

#Store config information in messages
messages['s'] = s
messages['e'] = e
messages['step'] = step
messages['max_tokens'] = max_tokens
messages['image_resize'] = image_resize
messages['s1'] = s1
messages['s3'] = s3
messages["API"] = API
messages['message'] = message

In [ ]:
def write_to_image(messages: dict) -> None:
  if not os.path.exists("outputs"):
    os.mkdir("outputs")

  with open(f"./outputs/{s}thru{e}with{step}-{API}.txt", 'w') as output_file:
    print(messages, file=output_file)

In [ ]:
# Go thru each, add the prompts to the messages dict
for i in range(1):
  #Google API
  if API == "GOOGLE":
    for i in range(s, e+1, step): # each of these defined at the top
      dir = f"{i:06d}"
      try:
        print(f"Reading {dir}")
        message = google_prompt_image(dir, image_resize=image_resize, s1=s1, s3=s3)
        messages[dir] = message
        write_to_image(messages)
      except Exception as e:
        print(f"Error: {e}")

  #OpenAI API

  if API == "OPENAI":
    for i in range(s, e+1, step): # each of these defined at the top
      dir = f"{i:06d}"
      try:
        print(f"Reading {dir}")
        #Clear context

        message = openai_prompt_image(dir, image_resize=image_resize, s1=s1, s3=s3)
        messages[dir] = message
        write_to_image(messages)
      except Exception as e:
        print(f"Error: {e}")

In [ ]:
messages